In [20]:
import pandas as pd

organizations_duplicates = pd.read_parquet("organizations_duplicates.parquet")
organizations_duplicates["unique_id"] = organizations_duplicates["OrganizationId"].astype(str)
organizations_duplicates.to_parquet("organizations_duplicates.parquet")

In [329]:
tmp_df = pd.read_parquet("organizations_duplicates.parquet")

In [331]:
sample_df = tmp_df.sample(frac=0.1)

In [333]:
sample_df.to_parquet("organizations_duplicates_sample.parquet")

In [1]:
import duckdb

In [21]:
con = duckdb.connect(database='test_duckdb.db', read_only=False)

In [22]:
con.execute("DROP TABLE IF EXISTS organizations")

In [23]:
con.execute("CREATE TABLE organizations AS SELECT * FROM 'organizations_duplicates.parquet';")

In [24]:
con.close()

In [12]:
from splink.duckdb.linker import DuckDBLinker

In [25]:
duckdb_con = duckdb.connect(database='test_duckdb.db', read_only=False)

In [212]:
duckdb_con.execute("SHOW ALL TABLES;").df()["name"].to_list()

['__splink__analyse_blocking_rule_f91acbedc',
 '__splink__df_comparison_vectors_7a66e589c',
 '__splink__df_concat_with_tf_a027c165f',
 '__splink__df_connected_components_df_8270de744',
 '__splink__df_connected_components_df_c5cc7b844',
 '__splink__df_neighbours_33a8a5b0f',
 '__splink__df_neighbours_f34ad8df9',
 '__splink__df_predict_b0109132e',
 '__splink__df_predict_da26584a1',
 '__splink__df_representatives_133fccfdc',
 '__splink__df_representatives_2_4e467e66d',
 '__splink__df_representatives_2_9dd60cc31',
 '__splink__df_representatives_eba4d9f14',
 'organizations',
 'test']

In [18]:
duckdb_con.close()

In [26]:
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl

settings = {
    "link_type": "dedupe_only",
    "comparisons": [
        cl.jaro_winkler_at_thresholds("Thoroughfare", [0.9, 0.8, 0.7]),
        ctl.name_comparison("LegalName")
    ],
    "blocking_rules_to_generate_predictions": [
        "l.NUTSId = r.NUTSId and substr(l.LegalName, 1,3) = substr(r.LegalName, 1,3)"
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": False,
}

In [27]:
linker = DuckDBLinker(input_table_or_tables="organizations", settings_dict=settings, connection=duckdb_con)

In [28]:
linker.count_num_comparisons_from_blocking_rule(
    "l.NUTSId = r.NUTSId and substr(l.LegalName, 1,3) = substr(r.LegalName, 1,3)")

114685

In [29]:
linker.estimate_u_using_random_sampling(max_pairs=1e6)

----- Estimating u probabilities using random sampling -----


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - Thoroughfare (no m values are trained).
    - LegalName (no m values are trained).


In [30]:
training_blocking_rule = "l.NUTSId = r.NUTSId"
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.NUTSId = r.NUTSId

Parameter estimates will be made for the following comparison(s):
    - Thoroughfare
    - LegalName

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Iteration 1: Largest change in params was 0.0255 in the m_probability of Thoroughfare, level `Jaro_winkler_similarity >= 0.9`
Iteration 2: Largest change in params was -0.101 in the m_probability of LegalName, level `Exact match LegalName`
Iteration 3: Largest change in params was -0.0513 in the m_probability of LegalName, level `Exact match LegalName`
Iteration 4: Largest change in params was -0.0719 in the m_probability of Thoroughfare, level `Exact match`
Iteration 5: Largest change in params was -0.173 in the m_probability of Thoroughfare, level `Exact match`
Iteration 6: Largest change in params was -0.17 in the m_probability of Thoroughfare, level `Exact match`
Iteration 7: Largest change in params was 0.263 in the m_probability of LegalName, level `All other comparisons`
Iteration 8: Largest change in params was 0.232 in the m_probability of LegalName, level `All other comparisons`
Iteration 9: Largest change in params was 0.07 in the m_probability of LegalName, level `All othe

In [33]:
settings = linker.save_model_to_json("saved_model_from_demo.json", overwrite=True)

In [214]:
linker = DuckDBLinker(input_table_or_tables="organizations", connection=duckdb_con)
linker.load_settings("saved_model_from_demo.json")

In [215]:
df_predictions = linker.predict(threshold_match_probability=0.8)

In [216]:
result_df = df_predictions.as_pandas_dataframe()

In [222]:
clusters_df = linker.cluster_pairwise_predictions_at_threshold(df_predictions, threshold_match_probability=0.8)

Completed iteration 1, root rows count 2
Completed iteration 2, root rows count 0


In [224]:
clusters_df = clusters_df.as_pandas_dataframe()
clusters_df


,cluster_id,OrganizationId,LegalName,AddressId,BusinessSize,CountryCode,PostCode,PostName,Thoroughfare,FullAddress,...,NUTS1,NUTS1Label,NUTS1LabelEng,NUTS2,NUTS2Label,NUTS2LabelEng,NUTS3,NUTS3Label,NUTS3LabelEng,unique_id
0,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,PERSONALHUSET STAFFING GROUP AS,http://data.europa.eu/a4g/resource/id_2023-S-0...,NaN,NOR,0159,Oslo,Grensen 17,NaN,...,NO0,Norge,Norge,NO08,Oslo og Viken,Oslo og Viken,NO081,Oslo,Oslo,http://data.europa.eu/a4g/resource/id_2023-S-0...
1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,"SIA ""VIA""",http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,LVA,LV-1026,Rīga,Starta iela 5,NaN,...,LV0,Latvija,Latvija,LV00,Latvija,Latvija,NaN,NaN,NaN,http://data.europa.eu/a4g/resource/id_2023-S-0...
2,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Huolto Hillman Oy,http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,FIN,NaN,Korppinen,NaN,NaN,...,FI1,Manner-Suomi,Manner-Suomi,FI19,Länsi-Suomi,Länsi-Suomi,FI193,Keski-Suomi,Keski-Suomi,http://data.europa.eu/a4g/resource/id_2023-S-0...
3,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,commune de toulon,http://data.europa.eu/a4g/resource/id_2023-S-0...,NaN,FRA,83056,Toulon,"avenue de la République, Cs 71407",NaN,...,FRL,Provence-Alpes-Côte d’Azur,Provence-Alpes-Côte d’Azur,FRL0,Provence-Alpes-Côte d’Azur,Provence-Alpes-Côte d’Azur,FRL05,Var,Var,http://data.europa.eu/a4g/resource/id_2023-S-0...
4,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,ENGIE SERVIZI SPA,http://data.europa.eu/a4g/resource/id_2023-S-0...,NaN,ITA,00144,ROMA,VIA G. RIBOTTA 31,NaN,...,ITI,Centro (IT),Centro (IT),ITI4,Lazio,Lazio,ITI43,Roma,Roma,http://data.europa.eu/a4g/resource/id_2023-S-0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32763,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Suffolk County Council,http://data.europa.eu/a4g/resource/id_2023-S-0...,NaN,GBR,NaN,Ipswich,NaN,NaN,...,UKH,East of England,East of England,UKH1,East Anglia,East Anglia,UKH14,Suffolk,Suffolk,http://data.europa.eu/a4g/resource/id_2023-S-0...
32764,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Wageningen University & Research,http://data.europa.eu/a4g/resource/id_2023-S-0...,NaN,NLD,6700 AB,Wageningen,Postbus 59,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://data.europa.eu/a4g/resource/id_2023-S-0...
32765,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,"UTE CIBERNOS CONSULTING, S.A.-OESIA NETWORKS, ...",http://data.europa.eu/a4g/resource/id_2023-S-0...,NaN,ESP,NaN,Madrid,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://data.europa.eu/a4g/resource/id_2023-S-0...
32766,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Przedsiębiorstwo Obrotu i Fabrykacji Artykułów...,http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,POL,60-573,Poznań,ul. Lindego 6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://data.europa.eu/a4g/resource/id_2023-S-0...


In [233]:
len(clusters_df["cluster_id"].unique())

23087

In [219]:
len(set(result_df["unique_id_l"].tolist()).union(set(result_df["unique_id_r"].tolist())))

13928

In [220]:
len(organizations_duplicates["unique_id"].unique())

32768

In [257]:
new_data = clusters_df.copy()
new_data["main_cluster_id"] = new_data["cluster_id"]
new_data.drop(columns=["cluster_id"], inplace=True)

In [264]:
new_data.columns

Index(['OrganizationId', 'LegalName', 'AddressId', 'BusinessSize',
       'CountryCode', 'PostCode', 'PostName', 'Thoroughfare', 'FullAddress',
       'NUTSId', 'NUTSLabel', 'NUTSLabelEng', 'NUTS0', 'NUTS0Label',
       'NUTS0LabelEng', 'NUTS1', 'NUTS1Label', 'NUTS1LabelEng', 'NUTS2',
       'NUTS2Label', 'NUTS2LabelEng', 'NUTS3', 'NUTS3Label', 'NUTS3LabelEng',
       'unique_id', 'main_cluster_id'],
      dtype='object')

In [305]:
new_settings = settings.copy()
new_settings["link_type"] = "link_only"
linker_only = DuckDBLinker(input_table_or_tables=[new_data, new_data.head(100)],
                           settings_dict=new_settings)

In [306]:
df_predict = linker_only.predict(threshold_match_probability=0.8)

In [307]:
df_predict_result = df_predict.as_pandas_dataframe()
df_predict_result

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,Thoroughfare_l,Thoroughfare_r,gamma_Thoroughfare,LegalName_l,LegalName_r,gamma_LegalName,NUTSId_l,NUTSId_r
0,7.159884,0.993056,__splink__input_table_0,__splink__input_table_1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Grensen 17,Grensen 17,4,PERSONALHUSET STAFFING GROUP AS,PERSONALHUSET STAFFING GROUP AS,4,NO081,NO081
1,7.159884,0.993056,__splink__input_table_0,__splink__input_table_1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,"avenue de la République, Cs 71407","avenue de la République, Cs 71407",4,commune de toulon,commune de toulon,4,FRL05,FRL05
2,7.159884,0.993056,__splink__input_table_0,__splink__input_table_1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,VIA G. RIBOTTA 31,VIA G. RIBOTTA 31,4,ENGIE SERVIZI SPA,ENGIE SERVIZI SPA,4,ITI43,ITI43
3,7.159884,0.993056,__splink__input_table_0,__splink__input_table_1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Birkenallee 51,Birkenallee 51,4,Gemeinde Bubenreuth,Gemeinde Bubenreuth,4,DE257,DE257
4,7.159884,0.993056,__splink__input_table_0,__splink__input_table_1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,place de l'hôtel de Ville,place de l'hôtel de Ville,4,ville de Perros-Guirec,ville de Perros-Guirec,4,FRH01,FRH01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,7.159884,0.993056,__splink__input_table_0,__splink__input_table_1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,"Strada Splaiul Independenţei, Nr. 202A","Strada Splaiul Independenţei, Nr. 202A",4,Autoritatea Nationala de Management al Calitat...,Autoritatea Nationala de Management al Calitat...,4,RO321,RO321
343,7.159884,0.993056,__splink__input_table_0,__splink__input_table_1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Podle trati 624/7,Podle trati 624/7,4,Alliance Healthcare s.r.o.,Alliance Healthcare s.r.o.,4,CZ010,CZ010
344,7.159884,0.993056,__splink__input_table_0,__splink__input_table_1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,"Strada Splaiul Independenţei, Nr. 202A","Strada Splaiul Independenţei, Nr. 202A",4,Autoritatea Nationala de Management al Calitat...,Autoritatea Nationala de Management al Calitat...,4,RO321,RO321
345,7.159884,0.993056,__splink__input_table_0,__splink__input_table_1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Huvudkontoret Röda vägen 1,Huvudkontoret Röda vägen 1,4,Trafikverket Myndighet,Trafikverket Myndighet,4,SE,SE


In [314]:
links = df_predict_result[['unique_id_l', 'unique_id_r', 'match_probability']]

In [324]:
links[["unique_id_l", "unique_id_l"]].rename(columns={"unique_id_l": "unique_id_r", "unique_id_r": "unique_id_l"})

,unique_id_l,unique_id_l
0,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
2,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
3,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
4,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
...,...,...
342,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
343,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
344,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
345,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...


In [325]:
tmp_a = pd.DataFrame([{"a":1, "b":2}, {"a":3, "b":4}])
tmp_b = pd.DataFrame([{"b": 2, "c": 5}, {"b": 5, "c": 6}])

In [328]:
tmp_a.merge(tmp_b, on="b", how="right")

,a,b,c
0,1.0,2,5
1,NaN,5,6


In [323]:
links.groupby(["unique_id_l"]).max().reset_index(level=0)

,unique_id_l,unique_id_r,match_probability
0,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,0.993056
1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,0.993056
2,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,0.993056
3,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,0.993056
4,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,0.993056
...,...,...,...
339,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,0.993056
340,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,0.993056
341,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,0.993056
342,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,0.993056


In [267]:
new_data[~new_data["unique_id"].isin(df_predict_result["unique_id_l"].unique().tolist())]

,OrganizationId,LegalName,AddressId,BusinessSize,CountryCode,PostCode,PostName,Thoroughfare,FullAddress,NUTSId,...,NUTS1Label,NUTS1LabelEng,NUTS2,NUTS2Label,NUTS2LabelEng,NUTS3,NUTS3Label,NUTS3LabelEng,unique_id,main_cluster_id
2,http://data.europa.eu/a4g/resource/id_2023-S-0...,Huolto Hillman Oy,http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,FIN,NaN,Korppinen,NaN,NaN,FI193,...,Manner-Suomi,Manner-Suomi,FI19,Länsi-Suomi,Länsi-Suomi,FI193,Keski-Suomi,Keski-Suomi,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
13,http://data.europa.eu/a4g/resource/id_2023-S-0...,UAB Jonavos šilumos tinklai,http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,LTU,NaN,jonava,NaN,NaN,LT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
18,http://data.europa.eu/a4g/resource/id_2023-S-0...,Fliesen Röhlich GmB,http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,DEU,90530,Wendelstein,NaN,NaN,DE25B,...,Bayern,Bayern,DE25,Mittelfranken,Mittelfranken,DE25B,Roth,Roth,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
24,http://data.europa.eu/a4g/resource/id_2023-S-0...,RINA CONSULTING SPA,http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,ITA,NaN,GENOVA,NaN,NaN,ITC33,...,Nord-Ovest,Nord-Ovest,ITC3,Liguria,Liguria,ITC33,Genova,Genova,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
28,http://data.europa.eu/a4g/resource/id_2023-S-0...,IMPLANTCAST POLSKA SPÓŁKA Z OGRANICZONĄ ODPOWI...,http://data.europa.eu/a4g/resource/id_2023-S-0...,NaN,POL,Warszawa,Warszawa,NaN,NaN,PL91,...,Makroregion województwo mazowieckie,Makroregion województwo mazowieckie,PL91,Warszawski stołeczny,Warszawski stołeczny,NaN,NaN,NaN,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32763,http://data.europa.eu/a4g/resource/id_2023-S-0...,Suffolk County Council,http://data.europa.eu/a4g/resource/id_2023-S-0...,NaN,GBR,NaN,Ipswich,NaN,NaN,UKH14,...,East of England,East of England,UKH1,East Anglia,East Anglia,UKH14,Suffolk,Suffolk,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
32764,http://data.europa.eu/a4g/resource/id_2023-S-0...,Wageningen University & Research,http://data.europa.eu/a4g/resource/id_2023-S-0...,NaN,NLD,6700 AB,Wageningen,Postbus 59,NaN,NL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
32765,http://data.europa.eu/a4g/resource/id_2023-S-0...,"UTE CIBERNOS CONSULTING, S.A.-OESIA NETWORKS, ...",http://data.europa.eu/a4g/resource/id_2023-S-0...,NaN,ESP,NaN,Madrid,NaN,NaN,ES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...
32766,http://data.europa.eu/a4g/resource/id_2023-S-0...,Przedsiębiorstwo Obrotu i Fabrykacji Artykułów...,http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,POL,60-573,Poznań,ul. Lindego 6,NaN,PL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...


In [278]:
len(new_data["LegalName"].value_counts().index)

21330

In [285]:
tmp_df = pd.DataFrame([{"a":"l1", "w": 0.1, "b":"c1"},
                       {"a":"l1", "w": 0.4, "b":"c1"},
                       {"a":"l1", "w": 0.3, "b":"c2"},
                       {"a":"l2", "w": 0.4, "b":"c3"}
                       ])

In [299]:
tmp_df.groupby(["a","b"]).sum().reset_index(level=[0,1]).groupby("a").max()

,b,w
a,,
l1,c2,0.5
l2,c3,0.4


In [49]:
clusters = linker.cluster_pairwise_predictions_at_threshold(df_predictions, threshold_match_probability=0.8)
result_clusters = clusters.as_pandas_dataframe()

Completed iteration 1, root rows count 2
Completed iteration 2, root rows count 0


In [53]:
clusters = result_clusters.groupby("cluster_id")

In [ ]:
def

In [ ]:
def clean_data(data: pd.DataFrame, )

In [122]:
def merge_cluster_to_single_representation(cluster_df: pd.DataFrame) -> dict:
    cluster_column_names = cluster_df.columns
    result_dict = {}
    for column_name in cluster_column_names:
        candidate_column_value = cluster_df[column_name].value_counts().sort_values(ascending=False).head(
            1).index.to_list()
        if len(candidate_column_value) == 1:
            result_dict[column_name] = candidate_column_value[0]
        else:
            result_dict[column_name] = None
    return result_dict

In [126]:
result_merged_clusters = [merge_cluster_to_single_representation(cluster) for cluster_id, cluster in clusters]
result_merged_clusters_df = pd.DataFrame(result_merged_clusters)

In [127]:
result_merged_clusters_df

,cluster_id,OrganizationId,LegalName,AddressId,BusinessSize,CountryCode,PostCode,PostName,Thoroughfare,FullAddress,...,NUTS1,NUTS1Label,NUTS1LabelEng,NUTS2,NUTS2Label,NUTS2LabelEng,NUTS3,NUTS3Label,NUTS3LabelEng,unique_id
0,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,ΖΑΡΑΪΔΩΝΗΣ ΧΡΗΣΤΟΣ,http://data.europa.eu/a4g/resource/id_2023-S-0...,None,GRC,19300,ΑΣΠΡΟΠΥΡΓΟΣ,ΘΕΣΗ ΚΟΥΤΑΛΑ,None,...,EL3,Αττική,Attiki,EL30,Aττική,Attiki,EL306,Δυτική Αττική,Dytiki Attiki,http://data.europa.eu/a4g/resource/id_2023-S-0...
1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,ΑΛΕΞΙΟΥ ΙΩΑΝΝΗΣ,http://data.europa.eu/a4g/resource/id_2023-S-0...,None,GRC,10683,ΑΘΗΝΑ,ΣΠΥΡΟΥ ΤΡΙΚΟΥΠΗ 44,None,...,EL3,Αττική,Attiki,EL30,Aττική,Attiki,EL303,Κεντρικός Τομέας Αθηνών,Kentrikos Tomeas Athinon,http://data.europa.eu/a4g/resource/id_2023-S-0...
2,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,ΕΝΩΣΗ ΜΑΚΕΔΟΝΙΑΣ ΑΝΩΝΥΜΗ ΜΕΤΑΦΟΡΙΚΗ ΕΤΑΙΡΕΙΑ Α...,http://data.europa.eu/a4g/resource/id_2023-S-0...,None,GRC,17778,ΤΑΥΡΟΣ,ΗΡΑΣ 4,None,...,EL3,Αττική,Attiki,EL30,Aττική,Attiki,EL303,Κεντρικός Τομέας Αθηνών,Kentrikos Tomeas Athinon,http://data.europa.eu/a4g/resource/id_2023-S-0...
3,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,ΗΛΕΚΤΡΟΝΙΚΟΣ ΕΘΝΙΚΟΣ ΦΟΡΕΑΣ ΚΟΙΝΩΝΙΚΗΣ ΑΣΦΑΛΙΣ...,http://data.europa.eu/a4g/resource/id_2023-S-0...,None,GRC,106 71,ΑΘΗΝΑ,ΑΚΑΔΗΜΙΑΣ 22,None,...,EL3,Αττική,Attiki,EL30,Aττική,Attiki,EL303,Κεντρικός Τομέας Αθηνών,Kentrikos Tomeas Athinon,http://data.europa.eu/a4g/resource/id_2023-S-0...
4,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,МУСАЛА- 88 ЕООД,http://data.europa.eu/a4g/resource/id_2023-S-0...,None,BGR,2781,с. Горно Краище,без улица №.0,None,...,BG4,Югозападна и Южна централна България,Yugozapadna i Yuzhna tsentralna Bulgaria,BG41,Югозападен,Yugozapaden,None,None,None,http://data.europa.eu/a4g/resource/id_2023-S-0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23082,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Djurambulansen Jonas Tysklind AB,http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,SWE,26994,BÅSTAD,Häljarpsvägen 83,None,...,SE2,Södra Sverige,Södra Sverige,SE23,Västsverige,Västsverige,None,None,None,http://data.europa.eu/a4g/resource/id_2023-S-0...
23083,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,SVENSKA HUNDSKYDDSFÖRENINGEN,http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,SWE,16838,BROMMA,Åkeshovs Gårdsväg 10,None,...,SE2,Södra Sverige,Södra Sverige,SE23,Västsverige,Västsverige,None,None,None,http://data.europa.eu/a4g/resource/id_2023-S-0...
23084,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,LÄNSSTYRELSEN I HALLANDS LÄN,http://data.europa.eu/a4g/resource/id_2023-S-0...,None,SWE,30186,HALMSTAD,Slottsgatan 2,None,...,SE2,Södra Sverige,Södra Sverige,SE23,Västsverige,Västsverige,SE231,Hallands län,Hallands län,http://data.europa.eu/a4g/resource/id_2023-S-0...
23085,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,AWP s.r.o.,http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,CZE,252 42,Jesenice u Prahy,Okružní 635,None,...,None,None,None,None,None,None,None,None,None,http://data.europa.eu/a4g/resource/id_2023-S-0...


In [143]:
result_merged_clusters_df["unique_id"] = result_merged_clusters_df["cluster_id"]

In [144]:
linker_only = DuckDBLinker(input_table_or_tables=[organizations_duplicates, result_merged_clusters_df],
                           settings_dict=new_settings)

In [145]:
df_predict = linker.predict(threshold_match_probability=0.8)

In [137]:
df_predict_result = df_predict.as_pandas_dataframe()

In [151]:
df_predict_result

,match_weight,match_probability,unique_id_l,unique_id_r,Thoroughfare_l,Thoroughfare_r,gamma_Thoroughfare,LegalName_l,LegalName_r,gamma_LegalName,NUTSId_l,NUTSId_r
0,7.159884,0.993056,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Kommunalförbundet Inköp Gävleborg Köpmannen 7,Kommunalförbundet Inköp Gävleborg Köpmannen 7,4,Inköp Gävleborg,Inköp Gävleborg,4,SE,SE
1,7.159884,0.993056,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Kommunalförbundet Inköp Gävleborg Köpmannen 7,Kommunalförbundet Inköp Gävleborg Köpmannen 7,4,Inköp Gävleborg,Inköp Gävleborg,4,SE,SE
2,7.159884,0.993056,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Kommunalförbundet Inköp Gävleborg Köpmannen 7,Kommunalförbundet Inköp Gävleborg Köpmannen 7,4,Inköp Gävleborg,Inköp Gävleborg,4,SE,SE
3,7.159884,0.993056,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,"B-dul 1 Mai, Nr. 6","B-dul 1 Mai, Nr. 6",4,REGIA NATIONALA A PADURILOR - ROMSILVA RA - DI...,REGIA NATIONALA A PADURILOR - ROMSILVA RA - DI...,4,RO215,RO215
4,7.159884,0.993056,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,"Strada 1 Decembrie 1918, Nr. 92","Strada 1 Decembrie 1918, Nr. 92",4,GETRANS BETON,GETRANS BETON,4,RO423,RO423
...,...,...,...,...,...,...,...,...,...,...,...,...
35904,7.159884,0.993056,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,2 rue des oeuillets,2 rue des oeuillets,4,semaas,semaas,4,FRI32,FRI32
35905,7.159884,0.993056,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,"Strada Splaiul Independenţei, Nr. 202A","Strada Splaiul Independenţei, Nr. 202A",4,Autoritatea Nationala de Management al Calitat...,Autoritatea Nationala de Management al Calitat...,4,RO321,RO321
35906,7.159884,0.993056,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,"Strada Splaiul Independenţei, Nr. 202A","Strada Splaiul Independenţei, Nr. 202A",4,Autoritatea Nationala de Management al Calitat...,Autoritatea Nationala de Management al Calitat...,4,RO321,RO321
35907,7.159884,0.993056,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Järve tn 34a,Järve tn 34a,4,Riigi Kaitseinvesteeringute Keskus,Riigi Kaitseinvesteeringute Keskus,4,EE,EE


In [131]:
new_settings = settings.copy()
new_settings["link_type"] = "link_only"

In [133]:
new_settings["link_type"] = "link_only"

In [157]:
result_merged_clusters_df

,cluster_id,OrganizationId,LegalName,AddressId,BusinessSize,CountryCode,PostCode,PostName,Thoroughfare,FullAddress,...,NUTS1,NUTS1Label,NUTS1LabelEng,NUTS2,NUTS2Label,NUTS2LabelEng,NUTS3,NUTS3Label,NUTS3LabelEng,unique_id
0,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,ΖΑΡΑΪΔΩΝΗΣ ΧΡΗΣΤΟΣ,http://data.europa.eu/a4g/resource/id_2023-S-0...,None,GRC,19300,ΑΣΠΡΟΠΥΡΓΟΣ,ΘΕΣΗ ΚΟΥΤΑΛΑ,None,...,EL3,Αττική,Attiki,EL30,Aττική,Attiki,EL306,Δυτική Αττική,Dytiki Attiki,http://data.europa.eu/a4g/resource/id_2023-S-0...
1,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,ΑΛΕΞΙΟΥ ΙΩΑΝΝΗΣ,http://data.europa.eu/a4g/resource/id_2023-S-0...,None,GRC,10683,ΑΘΗΝΑ,ΣΠΥΡΟΥ ΤΡΙΚΟΥΠΗ 44,None,...,EL3,Αττική,Attiki,EL30,Aττική,Attiki,EL303,Κεντρικός Τομέας Αθηνών,Kentrikos Tomeas Athinon,http://data.europa.eu/a4g/resource/id_2023-S-0...
2,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,ΕΝΩΣΗ ΜΑΚΕΔΟΝΙΑΣ ΑΝΩΝΥΜΗ ΜΕΤΑΦΟΡΙΚΗ ΕΤΑΙΡΕΙΑ Α...,http://data.europa.eu/a4g/resource/id_2023-S-0...,None,GRC,17778,ΤΑΥΡΟΣ,ΗΡΑΣ 4,None,...,EL3,Αττική,Attiki,EL30,Aττική,Attiki,EL303,Κεντρικός Τομέας Αθηνών,Kentrikos Tomeas Athinon,http://data.europa.eu/a4g/resource/id_2023-S-0...
3,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,ΗΛΕΚΤΡΟΝΙΚΟΣ ΕΘΝΙΚΟΣ ΦΟΡΕΑΣ ΚΟΙΝΩΝΙΚΗΣ ΑΣΦΑΛΙΣ...,http://data.europa.eu/a4g/resource/id_2023-S-0...,None,GRC,106 71,ΑΘΗΝΑ,ΑΚΑΔΗΜΙΑΣ 22,None,...,EL3,Αττική,Attiki,EL30,Aττική,Attiki,EL303,Κεντρικός Τομέας Αθηνών,Kentrikos Tomeas Athinon,http://data.europa.eu/a4g/resource/id_2023-S-0...
4,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,МУСАЛА- 88 ЕООД,http://data.europa.eu/a4g/resource/id_2023-S-0...,None,BGR,2781,с. Горно Краище,без улица №.0,None,...,BG4,Югозападна и Южна централна България,Yugozapadna i Yuzhna tsentralna Bulgaria,BG41,Югозападен,Yugozapaden,None,None,None,http://data.europa.eu/a4g/resource/id_2023-S-0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23082,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,Djurambulansen Jonas Tysklind AB,http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,SWE,26994,BÅSTAD,Häljarpsvägen 83,None,...,SE2,Södra Sverige,Södra Sverige,SE23,Västsverige,Västsverige,None,None,None,http://data.europa.eu/a4g/resource/id_2023-S-0...
23083,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,SVENSKA HUNDSKYDDSFÖRENINGEN,http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,SWE,16838,BROMMA,Åkeshovs Gårdsväg 10,None,...,SE2,Södra Sverige,Södra Sverige,SE23,Västsverige,Västsverige,None,None,None,http://data.europa.eu/a4g/resource/id_2023-S-0...
23084,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,LÄNSSTYRELSEN I HALLANDS LÄN,http://data.europa.eu/a4g/resource/id_2023-S-0...,None,SWE,30186,HALMSTAD,Slottsgatan 2,None,...,SE2,Södra Sverige,Södra Sverige,SE23,Västsverige,Västsverige,SE231,Hallands län,Hallands län,http://data.europa.eu/a4g/resource/id_2023-S-0...
23085,http://data.europa.eu/a4g/resource/id_2023-S-0...,http://data.europa.eu/a4g/resource/id_2023-S-0...,AWP s.r.o.,http://data.europa.eu/a4g/resource/id_2023-S-0...,sme,CZE,252 42,Jesenice u Prahy,Okružní 635,None,...,None,None,None,None,None,None,None,None,None,http://data.europa.eu/a4g/resource/id_2023-S-0...


In [183]:
from cleantext.sklearn import CleanTransformer

cleaner = CleanTransformer(
                        # fix_unicode=True,
                           # to_ascii=True,
                           # lower=True,
                           no_line_breaks=True,
                           normalize_whitespace=True
                           )

In [207]:
def preprocess_textual_columns(data: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess textual columns in a dataframe.
    :param data:
    :return:
    """
    cleaned_data = data.copy()
    for col in cleaned_data.columns:
        if cleaned_data[col].dtype == 'object':
            cleaned_data[col] = cleaned_data[col].str.casefold().str.replace(r"[^\w\s]",'', regex=True)
            cleaned_data[col] = cleaned_data[col].str.replace(r'\s+', ' ', regex=True)
    return cleaned_data


In [208]:
preprocessed_df = preprocess_textual_columns(result_merged_clusters_df)

In [209]:
preprocessed_df

,cluster_id,OrganizationId,LegalName,AddressId,BusinessSize,CountryCode,PostCode,PostName,Thoroughfare,FullAddress,...,NUTS1,NUTS1Label,NUTS1LabelEng,NUTS2,NUTS2Label,NUTS2LabelEng,NUTS3,NUTS3Label,NUTS3LabelEng,unique_id
0,httpdataeuropaeua4gresourceid_2023s085259678_c...,httpdataeuropaeua4gresourceid_2023s085259678_c...,ζαραϊδωνησ χρηστοσ,httpdataeuropaeua4gresourceid_2023s085259678_c...,None,grc,19300,ασπροπυργοσ,θεση κουταλα,None,...,el3,αττική,attiki,el30,aττική,attiki,el306,δυτική αττική,dytiki attiki,httpdataeuropaeua4gresourceid_2023s085259678_c...
1,httpdataeuropaeua4gresourceid_2023s085259678_c...,httpdataeuropaeua4gresourceid_2023s085259678_c...,αλεξιου ιωαννησ,httpdataeuropaeua4gresourceid_2023s085259678_c...,None,grc,10683,αθηνα,σπυρου τρικουπη 44,None,...,el3,αττική,attiki,el30,aττική,attiki,el303,κεντρικόσ τομέασ αθηνών,kentrikos tomeas athinon,httpdataeuropaeua4gresourceid_2023s085259678_c...
2,httpdataeuropaeua4gresourceid_2023s085259678_c...,httpdataeuropaeua4gresourceid_2023s085259678_c...,ενωση μακεδονιασ ανωνυμη μεταφορικη εταιρεια αμε,httpdataeuropaeua4gresourceid_2023s085259678_c...,None,grc,17778,ταυροσ,ηρασ 4,None,...,el3,αττική,attiki,el30,aττική,attiki,el303,κεντρικόσ τομέασ αθηνών,kentrikos tomeas athinon,httpdataeuropaeua4gresourceid_2023s085259678_c...
3,httpdataeuropaeua4gresourceid_2023s085259678_o...,httpdataeuropaeua4gresourceid_2023s089274506_o...,ηλεκτρονικοσ εθνικοσ φορεασ κοινωνικησ ασφαλισ...,httpdataeuropaeua4gresourceid_2023s089274506_o...,None,grc,106 71,αθηνα,ακαδημιασ 22,None,...,el3,αττική,attiki,el30,aττική,attiki,el303,κεντρικόσ τομέασ αθηνών,kentrikos tomeas athinon,httpdataeuropaeua4gresourceid_2023s085259678_o...
4,httpdataeuropaeua4gresourceid_2023s085259681_c...,httpdataeuropaeua4gresourceid_2023s094289272_c...,мусала 88 еоод,httpdataeuropaeua4gresourceid_2023s094289272_c...,None,bgr,2781,с горно краище,без улица 0,None,...,bg4,югозападна и южна централна българия,yugozapadna i yuzhna tsentralna bulgaria,bg41,югозападен,yugozapaden,None,None,None,httpdataeuropaeua4gresourceid_2023s085259681_c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23082,httpdataeuropaeua4gresourceid_2023s094291826_c...,httpdataeuropaeua4gresourceid_2023s094291826_c...,djurambulansen jonas tysklind ab,httpdataeuropaeua4gresourceid_2023s094291826_c...,sme,swe,26994,båstad,häljarpsvägen 83,None,...,se2,södra sverige,södra sverige,se23,västsverige,västsverige,None,None,None,httpdataeuropaeua4gresourceid_2023s094291826_c...
23083,httpdataeuropaeua4gresourceid_2023s094291826_c...,httpdataeuropaeua4gresourceid_2023s094291826_c...,svenska hundskyddsföreningen,httpdataeuropaeua4gresourceid_2023s094291826_c...,sme,swe,16838,bromma,åkeshovs gårdsväg 10,None,...,se2,södra sverige,södra sverige,se23,västsverige,västsverige,None,None,None,httpdataeuropaeua4gresourceid_2023s094291826_c...
23084,httpdataeuropaeua4gresourceid_2023s094291826_o...,httpdataeuropaeua4gresourceid_2023s094291826_o...,länsstyrelsen i hallands län,httpdataeuropaeua4gresourceid_2023s094291826_o...,None,swe,30186,halmstad,slottsgatan 2,None,...,se2,södra sverige,södra sverige,se23,västsverige,västsverige,se231,hallands län,hallands län,httpdataeuropaeua4gresourceid_2023s094291826_o...
23085,httpdataeuropaeua4gresourceid_2023s094291834_c...,httpdataeuropaeua4gresourceid_2023s094291834_c...,awp sro,httpdataeuropaeua4gresourceid_2023s094291834_c...,sme,cze,252 42,jesenice u prahy,okružní 635,None,...,None,None,None,None,None,None,None,None,None,httpdataeuropaeua4gresourceid_2023s094291834_c...


In [155]:
result_merged_clusters_df.memory_usage(index=True, deep=True).sum() / 1000000

46.794662

In [156]:
len(result_merged_clusters_df)

23087